# Ambulance Posting Project

## Exploratory Data Analysis



In [69]:
# Imports
import os
from urllib.request import urlretrieve
import urllib
import pandas as pd
import numpy as np

# Geo tools for handling coords and shapefiles
import geopandas as gpd
from shapely.geometry import Point

### Dataset

The dataset I will use for this project is the [Cincinnati Fire Incidents \(CAD\) \(including EMS: ALS/BLS\)](https://data.cincinnati-oh.gov/Safety/Cincinnati-Fire-Incidents-CAD-including-EMS-ALS-BL/vnsz-a3wp/about_data) dataset from the city of Cincinnati website's Open Data. The page for the dataset has an overview and preview of the dataset, as well as 3 pdfs that further describe the usages of each field.

First, I will download the data using their API endpoint. I can use SQL methods to limit the dataset. I will create a URL that limits the dataset to incidents from 2015 to 2024, and doesn't include columns that won't be useful to me. I can also exclude data that doesn't have latitude and longitude fields, since these will be essential for my project. Finally, I will make sure it's sorted by the create\_time\_incident column so I don't need to sort the data later.


In [70]:
# Define columns
dt_columns=['create_time_incident', 'dispatch_time_primary_unit', 'arrival_time_primary_unit', 'closed_time_incident']
all_columns = dt_columns + ['incident_type_id', 'incident_type_desc', 'disposition_text', 'cfd_incident_type', 'cfd_incident_type_group', 'address_x', 'latitude_x', 'longitude_x', 'neighborhood', 'beat']

In [71]:
# Base url for the query
cfd_url = "https://data.cincinnati-oh.gov/resource/vnsz-a3wp.csv?"

# SELECT columns
cfd_url += f"$query=SELECT "
cfd_url += ', '.join(all_columns)
    
# WHERE create_time_incident after 2015-01-01 00:00:00
cfd_url += " WHERE create_time_incident >= '2015-01-01T00:00:00' "

# AND create_time_incident before 2024-12-31 23:59:59
cfd_url += "AND create_time_incident <= '2024-12-31T23:59:59' "

# AND location fields are not NULL
cfd_url += "AND latitude_x IS NOT NULL "
cfd_url += "AND longitude_x IS NOT NULL "

# ORDER BY create_time_incident
cfd_url += " ORDER BY create_time_incident "

# LIMIT 1000000
cfd_url += "LIMIT 1000000"

cfd_url = urllib.parse.quote(cfd_url, safe="/:$'=?,", encoding='utf-8', errors='replace')

In [72]:
# Download the dataset if it is not already downloaded
cfd_path = "data/C1L6TY-6.csv"

if os.path.isfile(cfd_path):
    print("Dataset already exists, skipping download...")
else:
    print("Downloading dataset... ")
    urlretrieve(cfd_url, cfd_path)

df = pd.read_csv(cfd_path, parse_dates=dt_columns, date_format="%Y-%m-%dT%H:%M:%S.%f")
df.info()

Dataset already exists, skipping download...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857944 entries, 0 to 857943
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   create_time_incident        857944 non-null  datetime64[ns]
 1   dispatch_time_primary_unit  780365 non-null  datetime64[ns]
 2   arrival_time_primary_unit   736859 non-null  datetime64[ns]
 3   closed_time_incident        841542 non-null  datetime64[ns]
 4   incident_type_id            852372 non-null  object        
 5   incident_type_desc          725129 non-null  object        
 6   disposition_text            850693 non-null  object        
 7   cfd_incident_type           695394 non-null  object        
 8   cfd_incident_type_group     725795 non-null  object        
 9   address_x                   857792 non-null  object        
 10  latitude_x                  857944 non-null  float64       
 11  longitude_x                 857944 non-

The [CFD Data Visualization Exclusions.pdf](<CFD Data Visualization Exclusions.pdf>) from the dataset website documents values of certain fields that indicate entries where there was no response from the Cincinnati Fire Department. We can drop these rows from our dataset.


In [73]:
# Eliminate rows that documentation says did not result in a response from CFD
disp_text_no_response = ['AV: ADVISED','AV: ADVISED,CN: CANCEL','AV: ADVISED,CN: CANCEL,SBY: ST','AV: ADVISED,DEF: DEFAULT','AV: ADVISED,DUPF: DUPLICATE','AV: ADVISED,FADV: FIRE ADVISED','AV: ADVISED,TTY: TELETYPED','CAN:CANCEL','CANCEL INCIDENT','CN: CANCEL','CN: CANCEL,AV: ADVISED','CN: CANCEL,DEF: DEFAULT','CN: CANCEL,DUPF: DUPLICATE','CN: CANCEL,DUPF: DUPLICATE,FAD','CN: CANCEL,FADV: FIRE ADVISED','CN: CANCEL,FD: FIRE DISREGARD','CN: CANCEL,REASF: REASSIGNED','CN: CANCEL,SBY: STANDBY','CN: CANCEL,TTY: TELETYPED','CN: CANCEL,TTY: TELETYPED,DEF:','DUPF: DUPLICATE','DUPF: DUPLICATE,AV: ADVISED','DUPF: DUPLICATE,CN: CANCEL','DUPF: DUPLICATE,CN: CANCEL,DEF','DUPF: DUPLICATE,DEF: DEFAULT','DUPF: DUPLICATE,EMS: NO TRANSP','DUPF: DUPLICATE,FADV: FIRE ADV','DUPF: DUPLICATE,FALA: FIRE FAL','DUPF: DUPLICATE,GI: GOOD INTEN','DUPF: DUPLICATE,MAL: SYSTEM MA','DUPF: DUPLICATE,MEDT: MEDIC TR','DUPF: DUPLICATE,TTY: TELETYPED','DUPLICATE INCIDENT','TTY: TELETYPED']
cfd_inc_type_no_response = ['FIRE ADVISED NO DISPATCH','FIRE DRILL NO RESPONSE','FIRE SERVICE NO DISPATCH/NOT USED','INFORMATION TELETYPE NO DISPATCH','NOT USED']

df = df[~df['disposition_text'].isin(disp_text_no_response)]
print(f'Rows after dropping no response disposition_text values: {df.shape[0]}')

df = df[~df['cfd_incident_type_group'].isin(cfd_inc_type_no_response)]
print(f'Rows after dropping no response cfd_incident_type_group values: {df.shape[0]}')

Rows after dropping no response disposition_text values: 773322
Rows after dropping no response cfd_incident_type_group values: 772483


The [Cincinnati Fire Incidents \(CAD\) Data Dictionary.pdf](https://data.cincinnati-oh.gov/api/views/vnsz-a3wp/files/b149e911-467c-4828-9701-434e95cded55?download=true&filename=Cincinnati%20Fire%20Incidents%20(CAD)%20Data%20Dictionary.pdf) says that EMS responses For EMS responses have values ALS, BLS, NULL and MEDI in the cfd\_incident\_type field. The FIRE and OTHE fields are not EMS responses, so we can drop those.


In [74]:
# Get only EMS Responses
df = df[~df['cfd_incident_type'].isin(['FIRE', 'OTHE'])]
print(f'Rows after dropping non-EMS cfd_incident_type values: {df.shape[0]}')

Rows after dropping non-EMS cfd_incident_type values: 656794


We do not need to include rows where a unit was never dispatched, so we will only keep rows whose dispatch\_time\_primary\_unit value is not null.


In [75]:
# Drop incidents with no dispatch time
df = df[df['dispatch_time_primary_unit'].notnull()]
print(f'Rows after dropping incidents with no dispatch time: {df.shape[0]}')

Rows after dropping incidents with no dispatch time: 646294


Calls with a very short total duration \(from creation to close\) could not have had a true unit response, so we will try to eliminate some of these calls, as well.


In [76]:
# Drop calls that were closed 2 minutes or less after creation
df = df[abs((df['closed_time_incident'] - df['create_time_incident']).dt.total_seconds()) > 120]
print(f'Rows after dropping incidents that were closed soon after creation: {df.shape[0]}')

Rows after dropping incidents that were closed soon after creation: 631185


### Mutual Aid Calls

The majority of the incidents responded to are within the Cincinnati city limits, but some are not. These incident are usually mutual aid calls, which happen when a jurisdiction requests help from a neighbor. In these cases, surrounding areas are requesting aid from the Cincinnati Fire Department, and receiving a response unit. For the purposes of this project, we do not need to consider incidents that are outside of Cincinnati. The posting locations for ambulances around Cincinnati should be optimized for incidents that occur within the city boundaries. Although it is good to be able to respond to mutual aid calls quickly, these points are outliers that may distort the model.


In [77]:
display(df[['latitude_x','longitude_x']].describe())

,latitude_x,longitude_x
count,631185.000000,631185.000000
mean,39.141687,-84.516125
std,0.170007,0.069096
min,-45.997750,-114.997841
25%,39.114670,-84.559977
50%,39.137030,-84.516099
75%,39.160473,-84.484430
max,90.000660,-82.499190


By limiting the locations to points within city limits, we can also eliminate some extreme outliers that were probably errors, such as the one displayed below. The location 90.006, \-82.50002 is no where near Cincinnati, much less the given exit on I\-74.


In [80]:
display(df.loc[df['latitude_x'].idxmax()])

create_time_incident                        2020-01-24 02:15:46
dispatch_time_primary_unit                  2020-01-24 02:16:22
arrival_time_primary_unit                   2020-01-24 02:22:29
closed_time_incident                        2020-01-24 02:30:01
incident_type_id                                  ACCI-COMBINED
incident_type_desc            ACCIDENT WITH INJURIES (COMBINED)
disposition_text                              EMS: NO TRANSPORT
cfd_incident_type                                           NaN
cfd_incident_type_group                    ACCIDENT WITH INJURY
address_x                                         E I74 AT 17.9
latitude_x                                             90.00066
longitude_x                                           -82.50002
neighborhood                                                NaN
beat                                                       ST20
Name: 402566, dtype: object

I will use geopandas and a shapefile of the city of Cincinnati to filter out points that are not in the city boundaries.


In [81]:
# Load shapefile
shapefile_gdf = gpd.read_file("data/cincinnati_census_tracts/Social_Areas_2021_Cincinnati_City.shp")
shapefile_gdf.to_crs(epsg=4326, inplace=True)

# Create geometry from shapefile
geometry = [Point(xy) for xy in zip(df['longitude_x'], df['latitude_x'])]
gps_gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set CRS - GeoDataFrame has no CRS by default
gps_gdf.set_crs(shapefile_gdf.crs, inplace=True)

# Perform spatial join
points_within = gpd.sjoin(gps_gdf, shapefile_gdf, how='inner', predicate='within')

# Extract filtered GPS points
df = points_within[df.columns]
print(f'Rows after dropping incidents outside of Cincinnati area: {df.shape[0]}')

# Describe the location fields again for comparison
display(df[['latitude_x','longitude_x']].describe())

Rows after dropping incidents outside of Cincinnati area: 620186


,latitude_x,longitude_x
count,620186.000000,620186.000000
mean,39.140997,-84.516778
std,0.031814,0.056259
min,39.053282,-84.709411
25%,39.114461,-84.559884
50%,39.136475,-84.516288
75%,39.159433,-84.485257
max,39.220870,-84.369038
